In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split


# importer
df =pd.read_csv("../data/cleaned_data.csv")
#df = df[df["building_type"] != 'other-property']
# Selecting parameters to train our price prediction on
df_selection = df[[ 
    "price",              
    "living_area",
    "building_type",          
    "terrace",             
    "garden",              
    "swimming_pool",       
    "energy_class",
    "municipality",
    "province"]]

#df_selection = df_selection.replace(["apartment", "house"], [0, 1])
#Switching price to log_price


#Dropping missing values in living_area and price
df_selection = df_selection.dropna(subset=["living_area", "price"])
df_selection['log_price'] = df_selection['price'].apply(np.log10)
df_selection = df_selection.drop(columns = ['price'])
#Fill missing values in swimming_pool with zeros.
df_selection["swimming_pool"] = df_selection["swimming_pool"].fillna(0)


df_municipality = pd.get_dummies(df_selection[['municipality']])
df_province = pd.get_dummies(df_selection[['province']])

X = pd.concat([df_selection[["living_area",       
    "terrace",             
    "garden",              
    "swimming_pool",       
    "energy_class"]], df_municipality, df_province], axis=1)
y = df_selection['log_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

y_pred_full = 10**y_pred
y_test_full = 10**y_test

df_preds = pd.DataFrame({'Actual': y_test_full.squeeze(), 'Predicted': y_pred_full.squeeze()})
print(df_preds)



         Actual      Predicted
27850  209000.0  249509.303503
26178  220000.0  207350.789554
12171  135000.0  156784.725715
17394  255000.0  207377.361905
2130   195000.0  149679.136701
...         ...            ...
7926   200000.0  209308.227239
1630   130000.0  203867.605414
14559  215000.0  203005.229926
469    139000.0  202112.266506
17953  199000.0  200289.556613

[2132 rows x 2 columns]


In [28]:
from sklearn import metrics

print('R²:', metrics.r2_score(y_test, y_pred))

R²: 0.23234634121502784
